# Global FPGrowth Feature Importance Analysis

**Purpose:** Population-level frequent pattern mining for ML feature engineering  
**Updated:** November 23, 2025  
**Hardware:** Optimized for EC2 (32 cores, 1TB RAM)  
**Output:** `s3://pgxdatalake/gold/fpgrowth/global/{item_type}/`

## Key Features

✅ **Three Item Types** - Drugs, ICD codes, CPT codes  
✅ **Global Patterns** - Discovers patterns across all 5.7M patients  
✅ **ML Feature Engineering** - Creates encoding maps for CatBoost  
✅ **Association Rules** - Identifies co-occurrence relationships  
✅ **Memory Optimized** - Configurable support thresholds

## Methodology

For each item type (drug_name, icd_code, cpt_code):
1. Extract all unique items from cohort data
2. Create patient-level transactions (lists of items per patient)
3. Encode transactions into binary matrix
4. Run FP-Growth algorithm to find frequent itemsets
5. Generate association rules from frequent itemsets
6. Create encoding map for ML models
7. Save all outputs to S3

## Expected Runtime (EC2: 32 cores, 1TB RAM)

- **MIN_SUPPORT=0.01**: ~2-3 hours (12,783 drugs → 300-500 itemsets)
- **MIN_SUPPORT=0.005**: ~4-6 hours (more itemsets, more rules)
- **Total for all 3 types**: ~6-18 hours

## Data Scale

- **Total Events**: 947 million
- **Patients**: 5.7 million
- **Unique Drugs**: 12,783
- **Unique ICD Codes**: ~10,000
- **Unique CPT Codes**: ~5,000


## 1. Setup and Imports


In [1]:
import sys
import time
import json
import logging
from pathlib import Path
from typing import List, Dict
import pandas as pd
import boto3
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == '3_fpgrowth_analysis' else Path.cwd()
sys.path.insert(0, str(project_root))

from helpers_1997_13.duckdb_utils import get_duckdb_connection

print("✓ All imports successful")
print(f"✓ Project root: {project_root}")


✓ All imports successful
✓ Project root: /home/pgx3874/pgx-analysis


## 2. Configuration


In [ ]:
# =============================================================================
# EC2 CONFIGURATION (32 cores, 1TB RAM)
# =============================================================================

# FP-Growth parameters (quality-focused for ML features)
MIN_SUPPORT = 0.01       # Items must appear in 1% of patients (5.7M patients = 57K occurrences)
MIN_CONFIDENCE = 0.4     # 40% confidence - meaningful associations for CatBoost

# Item-specific thresholds (balance coverage vs quality)
MIN_CONFIDENCE_CPT = 0.5 # 50% confidence for CPT - strong procedure associations
MIN_SUPPORT_CPT = 0.02   # 2% support for CPT - focuses on common procedures

# Rule limits (quality over quantity)
MAX_RULES_PER_ITEM_TYPE = 5000  # Top 5000 rules by lift (for ML feature engineering)

# Target-focused rule mining (NEW!)
TARGET_FOCUSED = True  # Only generate rules that predict target outcomes
TARGET_ICD_CODES = ['F11.20', 'F11.21', 'F11.22', 'F11.23', 'F11.24', 'F11.25', 'F11.29']  # Opioid dependence codes
TARGET_HCG_LINES = [
    "P51 - ER Visits and Observation Care",
    "O11 - Emergency Room",
    "P33 - Urgent Care Visits"
]  # ED visits (HCG Line codes - matches phase2_event_processing.py)
TARGET_PREFIXES = ['TARGET_ICD:', 'TARGET_ED:']  # Prefixes for target items in transactions

# Item types to process
ITEM_TYPES = ['drug_name', 'icd_code', 'cpt_code']

# Paths
S3_OUTPUT_BASE = "s3://pgxdatalake/gold/fpgrowth/global"
LOCAL_DATA_PATH = project_root / "data"

# Setup logger with file output (prevents Jupyter rate limit issues)
logger = logging.getLogger('global_fpgrowth')
logger.setLevel(logging.INFO)
logger.handlers.clear()  # Clear any existing handlers

# File handler - full logs to file
log_file = project_root / "3_fpgrowth_analysis" / "global_fpgrowth_execution.log"
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)

# Console handler - only major milestones (prevents Jupyter rate limit)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.WARNING)  # Only warnings/errors to console
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(console_handler)

print(f"✓ Min Support (drug/ICD): {MIN_SUPPORT} (1% = ~57K patients)")
print(f"✓ Min Support (CPT): {MIN_SUPPORT_CPT} (2% = ~114K patients)")
print(f"✓ Min Confidence (drug/ICD): {MIN_CONFIDENCE} (40% - meaningful associations)")
print(f"✓ Min Confidence (CPT): {MIN_CONFIDENCE_CPT} (50% - strong procedure patterns)")
print(f"✓ Max Rules per Item Type: {MAX_RULES_PER_ITEM_TYPE:,} (top by lift)")
print(f"✓ Item Types: {ITEM_TYPES}")
print(f"✓ S3 Output: {S3_OUTPUT_BASE}")
print(f"✓ Local Data: {LOCAL_DATA_PATH}")
print(f"✓ Local Data Exists: {LOCAL_DATA_PATH.exists()}")
print(f"✓ Detailed logs → {log_file}")
print(f"✓ Console output: WARNING level only (check log file for progress)")
print("\n🎯 Quality-Focused Global Analysis:")
print("  - 5.7M patients = strong statistical power")
print("  - High confidence (40-50%) = meaningful patterns for CatBoost")
print("  - Top 5,000 rules per type = comprehensive yet manageable")
print("  - Encoding maps for all frequent items (for feature engineering)")
print(f"\n🎯 TARGET-FOCUSED RULE MINING: {'ENABLED' if TARGET_FOCUSED else 'DISABLED'}")
if TARGET_FOCUSED:
    print(f"  - Target ICD codes: {TARGET_ICD_CODES}")
    print(f"  - Target HCG lines (ED visits): {TARGET_HCG_LINES}")
    print("  - Only generates rules that PREDICT target outcomes")
    print("  - Example: {Metoprolol, Gabapentin} → {TARGET_ICD:OPIOID_DEPENDENCE}")
    print("  - Example: {99213: Office Visit, J0670: Morphine} → {TARGET_ED:EMERGENCY_DEPT}")
    print("  ✅ Focus on PREDICTIVE patterns across 5.7M patients")
    print("  ✅ Better CatBoost features (what predicts target?)")
    print("  ✅ Interpretable global patterns (risk factors)")


✓ Min Support: 0.01
✓ Min Confidence: 0.01
✓ Item Types: ['drug_name', 'icd_code', 'cpt_code']
✓ S3 Output: s3://pgxdatalake/gold/fpgrowth/global
✓ Local Data: /home/pgx3874/pgx-analysis/data
✓ Local Data Exists: True


## 3. Define Helper Functions


In [4]:
def extract_global_items(local_data_path: Path, item_type: str, logger: logging.Logger) -> List[str]:
    """Extract all unique items of specified type from local cohort data."""
    logger.info(f"Extracting global {item_type}s from local cohort data...")
    start_time = time.time()
    
    con = get_duckdb_connection(logger=logger)
    parquet_pattern = str(local_data_path / "**" / "cohort.parquet")
    
    if item_type == 'drug_name':
        query = f"""
        SELECT DISTINCT drug_name as item
        FROM read_parquet('{parquet_pattern}', hive_partitioning=1)
        WHERE drug_name IS NOT NULL AND drug_name != '' AND event_type = 'pharmacy'
        ORDER BY item
        """
    elif item_type == 'icd_code':
        query = f"""
        WITH all_icds AS (
            SELECT primary_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE primary_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT two_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE two_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT three_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE three_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT four_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE four_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT five_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE five_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
        )
        SELECT DISTINCT icd as item FROM all_icds WHERE icd != '' ORDER BY item
        """
    elif item_type == 'cpt_code':
        query = f"""
        SELECT DISTINCT procedure_code as item
        FROM read_parquet('{parquet_pattern}', hive_partitioning=1)
        WHERE procedure_code IS NOT NULL AND procedure_code != '' AND event_type = 'medical'
        ORDER BY item
        """
    else:
        raise ValueError(f"Unknown item_type: {item_type}")
    
    logger.info(f"Running query for {item_type}...")
    df = con.execute(query).df()
    con.close()
    items = df['item'].tolist()
    
    elapsed = time.time() - start_time
    logger.info(f"✓ Extracted {len(items):,} unique {item_type}s in {elapsed:.1f}s")
    return items


def create_global_transactions(local_data_path: Path, item_type: str, logger: logging.Logger) -> List[List[str]]:
    """Create patient-level transactions from local cohort data."""
    logger.info(f"Creating global {item_type} transactions...")
    start_time = time.time()
    
    con = get_duckdb_connection(logger=logger)
    parquet_pattern = str(local_data_path / "**" / "cohort.parquet")
    
    if item_type == 'drug_name':
        query = f"""
        SELECT mi_person_key, drug_name as item
        FROM read_parquet('{parquet_pattern}', hive_partitioning=1)
        WHERE drug_name IS NOT NULL AND drug_name != '' AND event_type = 'pharmacy'
        """
    elif item_type == 'icd_code':
        query = f"""
        WITH all_icds AS (
            SELECT mi_person_key, primary_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE primary_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT mi_person_key, two_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE two_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT mi_person_key, three_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE three_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT mi_person_key, four_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE four_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
            UNION ALL
            SELECT mi_person_key, five_icd_diagnosis_code as icd FROM read_parquet('{parquet_pattern}', hive_partitioning=1) 
            WHERE five_icd_diagnosis_code IS NOT NULL AND event_type = 'medical'
        )
        SELECT mi_person_key, icd as item FROM all_icds WHERE icd != ''
        """
    elif item_type == 'cpt_code':
        query = f"""
        SELECT mi_person_key, procedure_code as item
        FROM read_parquet('{parquet_pattern}', hive_partitioning=1)
        WHERE procedure_code IS NOT NULL AND procedure_code != '' AND event_type = 'medical'
        """
    else:
        raise ValueError(f"Unknown item_type: {item_type}")
    
    logger.info(f"Loading {item_type} events...")
    df = con.execute(query).df()
    con.close()
    
    logger.info(f"Grouping by patient...")
    transactions = (
        df.groupby('mi_person_key')['item']
        .apply(lambda x: sorted(set(x.tolist())))
        .tolist()
    )
    
    elapsed = time.time() - start_time
    logger.info(f"✓ Created {len(transactions):,} patient transactions in {elapsed:.1f}s")
    return transactions

print("✓ Helper functions defined")


✓ Helper functions defined


## 4. FP-Growth Processing

In [ ]:
def process_item_type(item_type, local_data_path, s3_output_base, min_support, min_confidence, logger):
    """Process a single item type: extract, encode, FP-Growth, save to S3."""
    logger.info(f"\n{'='*80}")
    logger.info(f"Processing {item_type.upper()}")
    logger.info(f"{'='*80}")
    overall_start = time.time()
    
    try:
        # Extract items
        items = extract_global_items(local_data_path, item_type, logger)
        
        # Create transactions
        transactions = create_global_transactions(local_data_path, item_type, logger)
        
        # Encode transactions
        logger.info(f"Encoding {len(transactions):,} transactions...")
        encode_start = time.time()
        te = TransactionEncoder()
        te_ary = te.fit(transactions).transform(transactions)
        df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
        encode_time = time.time() - encode_start
        logger.info(f"✓ Encoded to {df_encoded.shape} matrix in {encode_time:.1f}s")
        
        # Run FP-Growth (use item-specific parameters)
        actual_min_support = MIN_SUPPORT_CPT if item_type == 'cpt_code' else min_support
        logger.info(f"Running FP-Growth (min_support={actual_min_support})...")
        fpgrowth_start = time.time()
        itemsets = fpgrowth(df_encoded, min_support=actual_min_support, use_colnames=True)
        itemsets = itemsets.sort_values('support', ascending=False).reset_index(drop=True)
        fpgrowth_time = time.time() - fpgrowth_start
        logger.info(f"✓ Found {len(itemsets):,} frequent itemsets in {fpgrowth_time:.1f}s")
        
        # Generate association rules (with item-specific confidence and limits)
        actual_min_confidence = MIN_CONFIDENCE_CPT if item_type == 'cpt_code' else min_confidence
        logger.info(f"Generating association rules (min_confidence={actual_min_confidence})...")
        rules_start = time.time()
        
        try:
            rules = association_rules(itemsets, metric="confidence", min_threshold=actual_min_confidence)
            
            if len(rules) > 0:
                # Filter for target-focused rules (only rules predicting targets)
                if TARGET_FOCUSED:
                    pre_filter_count = len(rules)
                    # Keep only rules where consequent contains a target item
                    rules = rules[rules['consequents'].apply(
                        lambda x: any(item.startswith(tuple(TARGET_PREFIXES)) for item in x)
                    )]
                    logger.info(f"Target filter: {pre_filter_count:,} → {len(rules):,} rules (predicting target outcomes)")
                
                if len(rules) > 0:
                    # Sort by lift and limit to top N
                    rules = rules.sort_values('lift', ascending=False)
                    
                    if len(rules) > MAX_RULES_PER_ITEM_TYPE:
                        logger.info(f"Keeping top {MAX_RULES_PER_ITEM_TYPE:,} rules by lift (from {len(rules):,})")
                        rules = rules.head(MAX_RULES_PER_ITEM_TYPE)
                    else:
                        logger.info(f"Keeping all {len(rules):,} target-focused rules")
                    
                    rules = rules.reset_index(drop=True)
                else:
                    logger.info(f"No rules predict target outcomes")
            else:
                logger.info(f"No rules met confidence threshold of {actual_min_confidence}")
                
            rules_time = time.time() - rules_start
            logger.info(f"✓ Rule generation completed in {rules_time:.1f}s")
            
        except MemoryError as e:
            logger.error(f"MemoryError during rule generation")
            rules = pd.DataFrame()
            rules_time = time.time() - rules_start
        
        # Create encoding map
        encoding_map = {}
        for idx, row in itemsets.iterrows():
            if len(row['itemsets']) == 1:
                item = list(row['itemsets'])[0]
                encoding_map[item] = {'support': float(row['support']), 'rank': int(idx)}
        logger.info(f"✓ Created encoding map with {len(encoding_map):,} items")
        
        # Save to S3
        logger.info(f"Saving results to S3...")
        s3_client = boto3.client('s3')
        prefix = f"gold/fpgrowth/global/{item_type}"
        
        # Convert frozensets to lists
        itemsets_json = itemsets.copy()
        itemsets_json['itemsets'] = itemsets_json['itemsets'].apply(list)
        
        # Split rules by target type for separate files
        rules_by_target = {}
        if not rules.empty:
            rules_json = rules.copy()
            rules_json['antecedents'] = rules_json['antecedents'].apply(list)
            rules_json['consequents'] = rules_json['consequents'].apply(list)
            
            # Separate rules by target type
            rules_by_target['TARGET_ICD'] = rules_json[
                rules_json['consequents'].apply(lambda x: any('TARGET_ICD:' in str(item) for item in x))
            ]
            rules_by_target['TARGET_ED'] = rules_json[
                rules_json['consequents'].apply(lambda x: any('TARGET_ED:' in str(item) for item in x))
            ]
            rules_by_target['ALL'] = rules_json  # Keep all rules together too
            
            logger.info(f"Split rules: {len(rules_by_target['TARGET_ICD'])} ICD, {len(rules_by_target['TARGET_ED'])} ED")
        
        # Upload files
        s3_client.put_object(Bucket='pgxdatalake', Key=f"{prefix}/encoding_map.json", 
                            Body=json.dumps(encoding_map, indent=2))
        s3_client.put_object(Bucket='pgxdatalake', Key=f"{prefix}/itemsets.json", 
                            Body=itemsets_json.to_json(orient='records', indent=2))
        
        # Save rules by target type (separate files)
        if rules_by_target:
            for target_type, target_rules in rules_by_target.items():
                if not target_rules.empty:
                    s3_client.put_object(
                        Bucket='pgxdatalake', 
                        Key=f"{prefix}/rules_{target_type}.json",
                        Body=target_rules.to_json(orient='records', indent=2)
                    )
                    logger.info(f"✓ Saved {len(target_rules):,} {target_type} rules to S3")
        else:
            logger.info(f"✓ No rules to save (empty rules)")
        
        # Save metrics
        metrics = {
            'item_type': item_type, 
            'min_support': actual_min_support, 
            'min_confidence': actual_min_confidence,
            'max_rules_limit': MAX_RULES_PER_ITEM_TYPE,
            'rules_truncated': len(rules) == MAX_RULES_PER_ITEM_TYPE if len(rules) > 0 else False,
            'unique_items': len(items), 
            'total_transactions': len(transactions),
            'frequent_itemsets': len(itemsets), 
            'association_rules': len(rules),
            'rules_by_target': {
                'TARGET_ICD': len(rules_by_target.get('TARGET_ICD', pd.DataFrame())),
                'TARGET_ED': len(rules_by_target.get('TARGET_ED', pd.DataFrame()))
            } if rules_by_target else {'TARGET_ICD': 0, 'TARGET_ED': 0},
            'encoding_map_size': len(encoding_map),
            'target_focused': TARGET_FOCUSED,
            'target_icd_codes': TARGET_ICD_CODES if TARGET_FOCUSED else None,
            'target_hcg_lines': TARGET_HCG_LINES if TARGET_FOCUSED else None,
            'processing_time_seconds': {'total': time.time() - overall_start}
        }
        s3_client.put_object(Bucket='pgxdatalake', Key=f"{prefix}/metrics.json", 
                            Body=json.dumps(metrics, indent=2))
        
        logger.info(f"✓ {item_type.upper()} COMPLETE - {len(itemsets):,} itemsets, {len(rules):,} rules")
        return metrics
    except Exception as e:
        logger.error(f"✗ Failed: {e}", exc_info=True)
        return {'item_type': item_type, 'error': str(e)}

print("✓ process_item_type function defined")


✓ process_item_type function defined


## 5. Execute Analysis

Process all item types sequentially.


In [ ]:
print("="*80)
print("GLOBAL FPGROWTH ANALYSIS - START")
print("="*80)
print(f"Item types: {ITEM_TYPES}")
print(f"Min support: {MIN_SUPPORT}")
print(f"Min confidence: {MIN_CONFIDENCE}")
print(f"Detailed progress → Check log file")
print()

logger.info(f"\n{'='*80}")
logger.info(f"GLOBAL FPGROWTH ANALYSIS - START")
logger.info(f"{'='*80}")
logger.info(f"Item types: {ITEM_TYPES}")
logger.info(f"Min support: {MIN_SUPPORT}")
logger.info(f"Min confidence: {MIN_CONFIDENCE}")

# Helper function to check S3 existence
def check_s3_results_exist(s3_output_base: str, item_type: str) -> bool:
    """Check if results already exist in S3 (by checking for metrics.json)."""
    s3 = boto3.client('s3')
    key = f"gold/fpgrowth/global/{item_type}/metrics.json"
    try:
        s3.head_object(Bucket='pgxdatalake', Key=key)
        return True
    except:
        return False

overall_start = time.time()
all_metrics = []
skipped = 0

for idx, item_type in enumerate(ITEM_TYPES, 1):
    print(f"\n[{idx}/{len(ITEM_TYPES)}] Checking {item_type.upper()}...")
    
    # Check if already completed
    if check_s3_results_exist(S3_OUTPUT_BASE, item_type):
        print(f"⏭ {item_type.upper()} already exists in S3 - SKIPPING")
        logger.info(f"Skipping {item_type} - results already exist in S3")
        skipped += 1
        all_metrics.append({'item_type': item_type, 'status': 'skipped'})
        continue
    
    print(f"▶ Processing {item_type.upper()}...")
    item_start = time.time()
    
    metrics = process_item_type(
        item_type=item_type,
        local_data_path=LOCAL_DATA_PATH,
        s3_output_base=S3_OUTPUT_BASE,
        min_support=MIN_SUPPORT,
        min_confidence=MIN_CONFIDENCE,
        logger=logger
    )
    all_metrics.append(metrics)
    
    item_elapsed = time.time() - item_start
    if 'error' not in metrics:
        print(f"✓ {item_type.upper()} complete: {metrics['frequent_itemsets']:,} itemsets, "
              f"{metrics['association_rules']:,} rules ({item_elapsed/60:.1f}min)")
    else:
        print(f"✗ {item_type.upper()} failed: {metrics['error']}")

total_elapsed = time.time() - overall_start

print("\n" + "="*80)
print("GLOBAL FPGROWTH ANALYSIS - COMPLETE")
print("="*80)
print(f"Total time: {total_elapsed:.1f}s ({total_elapsed/60:.1f}min)")
print(f"Processed: {len([m for m in all_metrics if m.get('status') != 'skipped'])}")
print(f"Skipped: {skipped}")
print("\nResults Summary:")
for m in all_metrics:
    if m.get('status') == 'skipped':
        print(f"  ⏭ {m['item_type']}: SKIPPED (already in S3)")
    elif 'error' not in m:
        print(f"  ✓ {m['item_type']}: {m['frequent_itemsets']:,} itemsets, {m['association_rules']:,} rules")
    else:
        print(f"  ✗ {m['item_type']}: ERROR - {m['error']}")

logger.info(f"\n{'='*80}")
logger.info(f"GLOBAL FPGROWTH ANALYSIS - COMPLETE")
logger.info(f"{'='*80}")
logger.info(f"Total processing time: {total_elapsed:.1f}s ({total_elapsed/60:.1f}min)")
logger.info(f"\nResults Summary:")
for m in all_metrics:
    if 'error' not in m:
        logger.info(f"  {m['item_type']}: {m['frequent_itemsets']:,} itemsets, {m['association_rules']:,} rules")
    else:
        logger.info(f"  {m['item_type']}: ERROR - {m['error']}")

print("\n✓ Analysis complete!")


2025-11-23 21:50:40,601 - INFO - 
2025-11-23 21:50:40,602 - INFO - GLOBAL FPGROWTH ANALYSIS - START
2025-11-23 21:50:40,602 - INFO - ================================================================================
2025-11-23 21:50:40,602 - INFO - Item types: ['drug_name', 'icd_code', 'cpt_code']
2025-11-23 21:50:40,603 - INFO - Min support: 0.01
2025-11-23 21:50:40,603 - INFO - Min confidence: 0.01
2025-11-23 21:50:40,603 - INFO - 
2025-11-23 21:50:40,603 - INFO - Processing DRUG_NAME
2025-11-23 21:50:40,604 - INFO - ================================================================================
2025-11-23 21:50:40,604 - INFO - Extracting global drug_names from local cohort data...
2025-11-23 21:50:40,805 - INFO - ✅ Simple DuckDB connection created - 1 thread per worker (for multiprocessing)
2025-11-23 21:50:40,805 - INFO - Running query for drug_name...
2025-11-23 21:50:41,862 - INFO - ✓ Extracted 12,783 unique drug_names in 1.3s
2025-11-23 21:50:41,862 - INFO - Creating global drug_

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

2025-11-23 21:50:55,007 - INFO - Grouping by p